In [ ]:
import numpy as np
from astropy.io import fits
import moment_utils as mutil

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm, Normalize
SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


def plot_spectrum(spectrum, camera_wavelengths, image, pixel,
                  title=None, y_label=None, outfile=None):
    # convert wavelengths to doppler velocities
    LIGHT_SPEED = 3.0e10
    LINE_CM = .2600757633465  # CO line peak wavelength
    velocities = LIGHT_SPEED * (camera_wavelengths - LINE_CM) / LINE_CM
    # convert to km/s
    velocities /= 1.0e5

    plt.rcParams['figure.figsize'] = [24, 6]
    _, axes = plt.subplots(1, 2)
    spc, img = axes
    
    spc.plot(velocities, spectrum)
    spc.set_title(title)
    spc.set_xlabel("Velocity [km/s]")
    if y_label is None:
        y_label = "Specific Intensity [K]"
    spc.set_ylabel(y_label)
    
    img.pcolormesh(image, cmap='viridis', norm=LogNorm())
    img.set_aspect('equal', anchor='NW')
    x, y = pixel
    img.plot(x, y, 'rx')
    
    plt.show()
    
    if outfile:
        plt.savefig(outfile)
        print("Spectrum saved at {}".format(outfile))
        

def plot_moment_map(moment_map, moment, normed=False, scale_dimensions=None,
                    scale_units=None, outfile=None, colorscale=LogNorm(),
                    title=None, cbar_label=None):
    if scale_dimensions is None:
        scale_dimensions = moment_map.shape
    if scale_units is None:
        scale_units = "px"
    x_scale, y_scale = scale_dimensions
    x_lo, x_hi, y_lo, y_hi = -x_scale/2, x_scale/2, -y_scale/2, y_scale/2
    npix_y, npix_x = moment_map.shape
    x_range = np.linspace(x_lo, x_hi, npix_x)
    y_range = np.linspace(y_lo, y_hi, npix_y)
    
    plt.rcParams['figure.figsize'] = [12, 10]
    plt.pcolormesh(x_range, y_range, moment_map, cmap='viridis', norm=colorscale)
    cbar = plt.colorbar()
    plt.axis((x_lo, x_hi, y_lo, y_hi))
    plt.xlabel("X ({})".format(scale_units))
    plt.ylabel("Y ({})".format(scale_units))
    if normed:
        cbar.set_label(r'[(km/s)^{}]'.format(moment), rotation=90)
        plt.title(r'Normalized Integrated Temperature $\int T_v v^{} dv$'.format(moment))
    else:
        cbar.set_label(r'[K (km/s)^{}]'.format(1+moment), rotation=90)
        plt.title(r'Integrated Temperature $\int T_v v^{} dv$'.format(moment))
    if title is not None:
        plt.title(title)
    if cbar_label is not None:
        cbar.set_label(cbar_label, rotation=90)

    if outfile:
        plt.savefig(outfile)
        print("Moment map saved at {}".format(outfile))
        
    plt.show()

In [ ]:
full_mom0 = mutil.read_map("co_emission/full-mom0.fits")
tracer_mom0 = mutil.read_map("co_emission/tracer-mom0.fits")
nontracer_mom0 = mutil.read_map("co_emission/nontracer-mom0.fits")
obs_outflow_mom0 = mutil.read_map("co_emission/obs-outflow-mom0.fits")
obs_nonoutflow_mom0 = mutil.read_map("co_emission/obs-nonoutflow-mom0.fits")

nontracer_2ratio = nontracer_mom2/full_mom2
tracer_2ratio = tracer_mom2/full_mom2
obs_outflow_2ratio = obs_outflow_mom2/full_mom2
obs_nonoutflow_2ratio = obs_nonoutflow_mom2/full_mom2
obs_vs_tracer_2ratio = obs_outflow_mom2/tracer_mom2

Plot integrated maps

In [ ]:
maps = [full_mom0, tracer_mom0, nontracer_mom0, obs_outflow_mom0, obs_nonoutflow_mom0]
titles = ["Full 0-mom", "Tracer 0-mom", "Nontracer 0-mom", "Observer Outflow 0-mom",
          "Observer Nonoutflow 0-mom"]
outfiles = ["full-mom0", "tracer-mom0", "nontracer-mom0", "obs-outflow-mom0", "obs-nonoutflow-mom0"]
outfiles = ["co_emission/results/{}".format(fn) for fn in outfiles]

CM2PC = 3.2407792896664e-19
scale = 8e17*CM2PC, 8e17*CM2PC
for i in range(len(maps)):    
    plot_moment_map(maps[i], moment=0, scale_dimensions=scale, scale_units="pc",
                    colorscale=SymLogNorm(linthresh=1e-2), title=titles[i],
                    outfile=outfiles[i])

Plot 2nd-moment maps

In [ ]:
maps = [full_mom2, tracer_mom2, nontracer_mom2, obs_outflow_mom2, obs_nonoutflow_mom2]
titles = ["Full 2-mom", "Tracer 2-mom", "Nontracer 2-mom", "Observer Outflow 2-mom",
          "Observer Nonoutflow 2-mom"]
outfiles = ["full-mom2", "tracer-mom2", "nontracer-mom2", "obs-outflow-mom2", "obs-nonoutflow-mom2"]
outfiles = ["co_emission/results/{}".format(fn) for fn in outfiles]

CM2PC = 3.2407792896664e-19
scale = 8e17*CM2PC, 8e17*CM2PC
for i in range(len(maps)):    
    plot_moment_map(maps[i], moment=2, scale_dimensions=scale, scale_units="pc",
                    colorscale=SymLogNorm(linthresh=1e-2), title=titles[i],
                    outfile=outfiles[i])

Plot $v_{rms}$ maps

In [ ]:
maps = [full_vrms, tracer_vrms, nontracer_vrms, obs_outflow_vrms, obs_nonoutflow_vrms]
titles = [r"Full $v_{rms}$", r"Tracer $v_{rms}$", r"Nontracer $v_{rms}$",
          r"Observer Outflow $v_{rms}$", r"Observer Nonoutflow $v_{rms}$"]
outfiles = ["full-vrms", "tracer-vrms", "nontracer-vrms", "obs-outflow-vrms", "obs-nonoutflow-vrms"]
outfiles = ["co_emission/results/{}".format(fn) for fn in outfiles]

CM2PC = 3.2407792896664e-19
scale = 8e17*CM2PC, 8e17*CM2PC
for i in range(len(maps)):    
    plot_moment_map(maps[i], moment=2, scale_dimensions=scale, scale_units="pc",
                    colorscale=Normalize(), title=titles[i],
                    cbar_label="km/s", outfile=outfiles[i])

Plot ratio maps

In [ ]:
maps = [tracer_2ratio, nontracer_2ratio, obs_outflow_2ratio, obs_nonoutflow_2ratio, obs_vs_tracer_2ratio]
titles = ["Tracer/Full 2-mom ratio", "Nontracer/Full 2-mom ratio",
          "ObserverOutflow/Full 2-mom ratio", "ObserverNonoutflow/Full 2-mom ratio",
          "ObserverOutflow/Tracer 2-mom ratio"]
outfiles = ["ratio-tracer-mom2", "ratio-nontracer-mom2", "ratio-obs-outflow-mom2",
            "ratio-obs-nonoutflow-mom2", "ratio-obs-vs-tracer-mom2"]
outfiles = ["co_emission/results/{}".format(fn) for fn in outfiles]

CM2PC = 3.2407792896664e-19
scale = 8e17*CM2PC, 8e17*CM2PC
for i in range(len(maps)):    
    plot_moment_map(maps[i], moment=2, scale_dimensions=scale, scale_units="pc",
                    colorscale=Normalize(), title=titles[i],
                    cbar_label="[unitless]", outfile=outfiles[i])

Plot spectra

In [ ]:
_, camera_wavelengths, temp_cube = mutil.get_temp_cube("co_emission/full.fits")
full_mom0 = mutil.read_map("co_emission/full-mom0.fits")

In [ ]:
px = (100, 150)
spectrum = mutil.create_spectrum(temp_cube, pixel=px)
plot_spectrum(spectrum, camera_wavelengths, image=full_mom0, pixel=px, title="Spectrum @ {}".format(px))